# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 20:06:16,624 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
! hadoop fs -copyFromLocal clickstream.csv

In [3]:
! hadoop fs -ls -h 

Found 2 items
drwxr-xr-x   - jovyan supergroup          0 2024-10-27 20:06 .sparkStaging
-rw-r--r--   1 jovyan supergroup     30.7 M 2024-10-27 20:06 clickstream.csv


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

df = se.read.option("delimiter", "\t").csv("clickstream.csv", header=True)

In [11]:
from pyspark.sql.window import Window

error_df = df.filter(F.lower(df.event_type).contains("error"))
error_times = error_df.groupBy("session_id", "user_id") \
    .agg(F.min("timestamp").alias("error_timestamp"))

clean_df = (
    df
    .join(error_times, on=["session_id", "user_id"], how="left")
    .withColumn("valid", F.when(F.col("error_timestamp").isNull() | 
                                (F.col("timestamp") < F.col("error_timestamp")), 1).otherwise(0)) 
    .filter(F.col("valid") == 1) 
    .drop("valid", "error_timestamp")
)


In [16]:
window_spec = Window.partitionBy("session_id", "user_id").orderBy("timestamp")

clickstream_with_prev_df = clean_df.withColumn(
    "prev_page", F.lag("event_page").over(window_spec)
)

filtered_clean_df = clickstream_with_prev_df.filter(
    (F.col("event_page") != F.col("prev_page")) | F.col("prev_page").isNull()
)

routes_df = filtered_clean_df.groupBy("session_id", "user_id").agg(
    F.concat_ws("-", F.collect_list("event_page")).alias("route")
).orderBy("session_id")

routes_df.show()

+----------+-------+--------------------+
|session_id|user_id|               route|
+----------+-------+--------------------+
|         0|    412|main-digital-news...|
|         0|   4624|main-tariffs-bonu...|
|         0|   4488|main-bonus-rabota...|
|         0|   2156|        main-tariffs|
|         0|   2536| main-tariffs-online|
|         1|    322|main-news-vklad-i...|
|         1|   2043|                main|
|         1|   2620|    main-rabota-news|
|         1|    178|main-internet-tar...|
|         1|   1995|main-archive-tari...|
|         1|   1374|main-internet-new...|
|         1|    170|main-bonus-main-i...|
|        10|   4214|           main-news|
|        10|   4026|          main-vklad|
|        10|   2936|main-internet-onl...|
|        10|    170|main-bonus-vklad-...|
|        10|   1492|main-tariffs-bonu...|
|        10|   2556|main-news-interne...|
|        10|   2043|main-tariffs-news...|
|        10|   2564|main-tariffs-rabo...|
+----------+-------+--------------

In [17]:
routes_df = routes_df.groupBy("route") \
    .agg(F.count("session_id").alias("count")) \
    .orderBy(F.desc("count"))
top_30_routes_df = routes_df.limit(30)

In [18]:
top_30_routes_df.collect()

[Row(route='main', count=8184),
 Row(route='main-archive', count=1113),
 Row(route='main-rabota', count=1046),
 Row(route='main-internet', count=897),
 Row(route='main-bonus', count=870),
 Row(route='main-news', count=769),
 Row(route='main-tariffs', count=677),
 Row(route='main-online', count=586),
 Row(route='main-vklad', count=518),
 Row(route='main-rabota-archive', count=170),
 Row(route='main-archive-rabota', count=167),
 Row(route='main-bonus-archive', count=143),
 Row(route='main-rabota-bonus', count=139),
 Row(route='main-news-rabota', count=135),
 Row(route='main-bonus-rabota', count=135),
 Row(route='main-archive-internet', count=132),
 Row(route='main-rabota-news', count=130),
 Row(route='main-internet-rabota', count=129),
 Row(route='main-archive-news', count=126),
 Row(route='main-rabota-internet', count=124),
 Row(route='main-internet-archive', count=123),
 Row(route='main-archive-bonus', count=117),
 Row(route='main-internet-bonus', count=115),
 Row(route='main-tariffs-i

In [19]:
top_30_routes_df

DataFrame[route: string, count: bigint]

In [20]:
top_30_routes_df.write.json('result.json', mode='overwrite')

In [21]:
! hadoop fs -ls -h 

Found 3 items
drwxr-xr-x   - jovyan supergroup          0 2024-10-27 20:08 .sparkStaging
-rw-r--r--   1 jovyan supergroup     30.7 M 2024-10-27 20:06 clickstream.csv
drwxr-xr-x   - jovyan supergroup          0 2024-10-27 20:13 result.json


In [28]:
pandas_df = top_30_routes_df.toPandas()
result_dict = pandas_df.set_index('route').T.to_dict('records')
simple_dict = {row['route']: row['count'] for row in pandas_df.to_dict('records')}

In [31]:
import json
with open("final_result.json", "w") as json_file:
    json.dump(simple_dict, json_file, indent=4)


In [35]:
simple_dict

{'main': 8184,
 'main-archive': 1113,
 'main-rabota': 1046,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 586,
 'main-vklad': 518,
 'main-rabota-archive': 170,
 'main-archive-rabota': 167,
 'main-bonus-archive': 143,
 'main-rabota-bonus': 139,
 'main-news-rabota': 135,
 'main-bonus-rabota': 135,
 'main-archive-internet': 132,
 'main-rabota-news': 130,
 'main-internet-rabota': 129,
 'main-archive-news': 126,
 'main-rabota-internet': 124,
 'main-internet-archive': 123,
 'main-archive-bonus': 117,
 'main-internet-bonus': 115,
 'main-tariffs-internet': 114,
 'main-news-archive': 113,
 'main-news-internet': 109,
 'main-archive-tariffs': 104,
 'main-internet-news': 103,
 'main-tariffs-archive': 103,
 'main-rabota-main': 94}

In [37]:
!curl -F file=@final_result.json 51.250.123.136:80/MDS-LSML1/rmnv_igor/w6/2

0.8999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Wrong main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!
Wrong main-archive-rabota answer!
Wrong main-bonus-archive answer!
Wrong main-rabota-bonus answer!
Wrong main-news-rabota answer!
Wrong main-bonus-rabota answer!
Wrong main-archive-internet answer!
Wrong main-rabota-news answer!
Wrong main-internet-rabota answer!
Wrong main-archive-news answer!
Wrong main-rabota-internet answer!
Wrong main-internet-archive answer!
Wrong main-archive-bonus answer!
Wrong main-internet-bonus answer!
Wrong main-tariffs-internet answer!
Wrong main-news-archive answer!
Wrong main-news-internet answer!
Wrong main-archive-tariffs answer!
Wrong main-internet-news answer!
Wrong main-tariffs-archive answer!
Wrong main-rabota-main answer!


# 2

In [38]:
df.createOrReplaceTempView("clickstream")

valid_routes_sql = """
WITH ErrorSessions AS (
    SELECT session_id, user_id, MIN(timestamp) AS error_time
    FROM clickstream
    WHERE LOWER(event_type) LIKE '%error%'
    GROUP BY session_id, user_id
),
FilteredData AS (
    SELECT c.*
    FROM clickstream c
    LEFT JOIN ErrorSessions e 
         ON c.session_id = e.session_id
            and c.user_id = e.user_id
    WHERE e.error_time IS NULL OR c.timestamp < e.error_time
)
SELECT 
    session_id,
    user_id,
    CONCAT_WS('-', COLLECT_LIST(event_page)) AS route
FROM (
    SELECT session_id, user_id, event_page, timestamp,
           LEAD(event_page) OVER (PARTITION BY session_id, user_id ORDER BY timestamp) AS prev_page
    FROM FilteredData
) AS filtered
WHERE event_page != prev_page OR prev_page IS NULL
GROUP BY session_id, user_id
ORDER BY session_id, user_id;
"""

valid_routes_df = se.sql(valid_routes_sql)


In [44]:
route_counts_df = valid_routes_df.groupBy("route").agg(F.count("session_id").alias("count")) \
                                  .orderBy(F.desc("count"))

top_routes_df2 = route_counts_df.limit(30)

In [45]:
pandas_df2 = top_routes_df2.toPandas()
result_dict2 = pandas_df2.set_index('route').T.to_dict('records')
simple_dict2 = {row['route']: row['count'] for row in pandas_df2.to_dict('records')}

In [46]:
import json
with open("final_result2.json", "w") as json_file:
    json.dump(simple_dict2, json_file, indent=4)


In [47]:
!curl -F file=@final_result2.json 51.250.123.136:80/MDS-LSML1/rmnv_igor/w6/2

0.8999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Wrong main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!
Wrong main-archive-rabota answer!
Wrong main-bonus-archive answer!
Wrong main-rabota-bonus answer!
Wrong main-news-rabota answer!
Wrong main-bonus-rabota answer!
Wrong main-archive-internet answer!
Wrong main-rabota-news answer!
Wrong main-internet-rabota answer!
Wrong main-archive-news answer!
Wrong main-rabota-internet answer!
Wrong main-internet-archive answer!
Wrong main-archive-bonus answer!
Wrong main-internet-bonus answer!
Wrong main-tariffs-internet answer!
Wrong main-news-archive answer!
Wrong main-news-internet answer!
Wrong main-archive-tariffs answer!
Wrong main-internet-news answer!
Wrong main-tariffs-archive answer!
Wrong main-rabota-main answer!
